### This notebook uses Keith's diagnostic bash script to run ILAMB (without having to CMORize data first), generates paths, and submits the ilamb run script

###### To Do

- rosie_diagnostics works. 

- look at output locally
- get output onto website
- make this script more generic
- comparision bash scripts are in 
-/glade/work/rfisher/diagnostics_ilamb_standalone/runs/clm5_sp



In [1]:
# Import all normal libraries. Likely some of these aren't needed
import os
import netCDF4 as nc4
import sys
import shutil
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
from matplotlib import pyplot as plt
#import datetime

##### Make function to create directories.

In [2]:
def make_directory(fileroot):   
    if(os.path.isdir(fileroot)):
        print('dir exists:'+fileroot)
    else:
        os.mkdir(fileroot)
        print('made: '+fileroot)

##### Make path names

In [3]:
#paths
USER='rfisher'

# github repo for these scripts
fates_global_cal_path='/glade/u/home/'+USER+'/fates-global-cal'

# where are the ILAMB data?
ilamb_dir  ='/glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all'
ilamb_models= ilamb_dir+'/MODELS/' 

#ILAMB config file
config_file='ilamb_CLMFATES_SP.cfg'
config_path=fates_global_cal_path+'/ilamb_scripts/ncar_ilambscripts/'+config_file

#model setup path
model_setup_file='model_setup_fatesrtm.txt'
model_setup_path = fates_global_cal_path + '/ilamb_scripts/model_setup_files/'+ model_setup_file

#ILAMB regions file
ilamb_regions='/glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/DATA/regions/LandRegions.nc'

#### Keith's setup
#model_setup_file='keith_model_setup.txt'
#model_setup_path = ilamb_scripts_dir + model_setup_file
#model_setup_path ='/glade/work/rfisher/diagnostics_ilamb_standalone/runs/clm5_sp/model_setup_fatesrtm.txt'
list_models='RTM_ens3_case_1' 'RTM_ens3_case_2' 'RTM_ens3_case_3'
print('model config file='+ config_path)
print('model setup path='+model_setup_path)

model config file=/glade/u/home/rfisher/fates-global-cal/ilamb_scripts/ncar_ilambscripts/ilamb_CLMFATES_SP.cfg
model setup path=/glade/u/home/rfisher/fates-global-cal/ilamb_scripts/model_setup_files/model_setup_fatesrtm.txt


### ilamb diagnostic bash script


In [ ]:
%%bash -s "" "$config_path" "$model_setup_path"

config_path=$1  
model_setup_path=$2

#PBS -N ilamb_diagnostics*
#PBS -q regular
#PBS -l select=4:ncpus=2:mpiprocs=2:mem=109GB
#PBS -l walltime=12:00:00
#PBS -A P93300041
#PBS -j oe
#PBS -k eod
#PBS -m abe
#PBS -M oleson@ucar.edu

#source /etc/profile.d/modules.sh

echo $config_path
echo $model_setup_path

export caseid=clm5_sp

#conda init bash
source /glade/work/oleson/miniconda3/etc/profile.d/conda.sh

export MPI_UNBUFFERED_STDIO=true
export TMPDIR=$TMPDIR
mkdir -p $TMPDIR

conda activate ilamb

#today="$(date '+%Y%m%d-%H%M%S')"
#log_filename=/glade/work/oleson/diagnostics_ilamb_standalone/runs/$caseid/logs/ilamb_diagnostics.log.$today
#mkdir -p /glade/work/oleson/diagnostics_ilamb_standalone/runs/$caseid/logs

export MPLBACKEND=Agg

export ILAMB_ROOT=/glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all

cp /glade/work/oleson/ilamb_stage-next-version/demo/ilamb_nohoff_final_CLM.cfg .

# To check peak memory usage
#module load peak_memusage
#export MPI_SHEPHERD=true
#mpiexec_mpt peak_memusage.exe dplace -s 1 /glade/work/oleson/ilamb_standalone/bin/ilamb-run  --config ./ilamb_nohoff_final_CLM_SP.cfg --model_root /glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/MODELS/  --models R15 PPE PPEnocrop --build_dir /glade/work/oleson/diagnostics_ilamb_standalone/runs/$caseid/_build_PPE2nocrop --define_regions /glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/DATA/regions/LandRegions.nc --regions global --model_setup /glade/work/oleson/diagnostics_ilamb_standalone/runs/$caseid/model_setup.txt --filter .clm2.h0. >> ${log_filename} 2>&1

# Normal submission
#mpiexec_mpt dplace -s 1 /glade/work/rfisher/ilamb/ILAMB/bin/ilamb-run  --config ./ilamb_nohoff_final_CLM.cfg --model_root /glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/MODELS/  --models CLM50 CTSM51  --define_regions /glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/DATA/regions/LandRegions.nc --regions global --model_setup /glade/work/oleson/diagnostics_ilamb_standalone/runs/$caseid/model_setup.txt --filter .clm2.h0.

echo "submitting ilamb"
#rosie modifications
mpiexec_mpt /glade/work/rfisher/ilamb/ILAMB/bin/ilamb-run  \
--config /glade/u/home/rfisher/fates-global-cal/ilamb_scripts/ncar_ilambscripts/ilamb_CLMFATES_SP.cfg \
--model_root /glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/MODELS/   \
--models  RTM_ens3_case_1 RTM_ens3_case_2 RTM_ens3_case_3 \
--build_dir /glade/work/rfisher/diagnostics_ilamb_standalone/runs/$caseid/_build_clm5_sp \
--model_setup /glade/u/home/rfisher/fates-global-cal/ilamb_scripts/model_setup_files/model_setup_fatesrtm.txt \
--filter .clm2.h0.


#>> ${log_filename} 2>&1


In [8]:
%%bash -s "" "$config_path" "$model_setup_path"

config_path=$1  
model_setup_path=$2

#PBS -N ilamb_diagnostics*
#PBS -q regular
#PBS -l select=4:ncpus=2:mpiprocs=2:mem=109GB
#PBS -l walltime=12:00:00
#PBS -A P93300041
#PBS -j oe
#PBS -k eod
#PBS -m abe
#PBS -M oleson@ucar.edu

#source /etc/profile.d/modules.sh

echo $config_path
echo $model_setup_path

export caseid=clm5_sp

#conda init bash
source /glade/work/oleson/miniconda3/etc/profile.d/conda.sh

export MPI_UNBUFFERED_STDIO=true
export TMPDIR=$TMPDIR
mkdir -p $TMPDIR

conda activate ilamb

#today="$(date '+%Y%m%d-%H%M%S')"
#log_filename=/glade/work/oleson/diagnostics_ilamb_standalone/runs/$caseid/logs/ilamb_diagnostics.log.$today
#mkdir -p /glade/work/oleson/diagnostics_ilamb_standalone/runs/$caseid/logs

export MPLBACKEND=Agg

export ILAMB_ROOT=/glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all

cp /glade/work/oleson/ilamb_stage-next-version/demo/ilamb_nohoff_final_CLM.cfg .

# Normal submission
#mpiexec_mpt dplace -s 1 /glade/work/rfisher/ilamb/ILAMB/bin/ilamb-run  --config ./ilamb_nohoff_final_CLM.cfg --model_root /glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/MODELS/  --models CLM50 CTSM51  --define_regions /glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/DATA/regions/LandRegions.nc --regions global --model_setup /glade/work/oleson/diagnostics_ilamb_standalone/runs/$caseid/model_setup.txt --filter .clm2.h0.

echo "submitting ilamb"
#rosie modifications
mpiexec_mpt dplace -s 1 /glade/u/home/rfisher/fates-global-cal/ilamb_scripts/ilamb-run --config config_files/ilamb_CLMFATES_SP.cfg \
--model_root /glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/MODELS/  \
--models  RTM_ens4_case_1 RTM_ens3_case_1 RTM_ens3_case_10 \
--build_dir /glade/u/home/rfisher/fates-global-cal/ilamb_scripts/fates_rtm/_build \
--define_regions /glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all/DATA/regions/LandRegions.nc --regions global \
--model_setup /glade/u/home/rfisher/fates-global-cal/ilamb_scripts/model_setup_files/model_setup_fatesrtm.txt \
--filter  .clm2.h0. \
--confrontation Albedo/CERES
#>> ${log_filename} 2>&1



/glade/u/home/rfisher/fates-global-cal/ilamb_scripts/ncar_ilambscripts/ilamb_CLMFATES_SP.cfg
submitting ilamb

Setting up model results from /glade/u/home/rfisher/fates-global-cal/ilamb_scripts/model_setup_files/model_setup_fatesrtm.txt

                                  RTM_ens4_case_1
                                  RTM_ens3_case_1
                                 RTM_ens3_case_10

Parsing config file config_files/ilamb_CLMFATES_SP.cfg...

                 GrossPrimaryProductivity/Fluxnet Initialized
                    GrossPrimaryProductivity/GBAF Initialized
                         Evapotranspiration/GLEAM Initialized
                         Evapotranspiration/MODIS Initialized
                         EvaporativeFraction/GBAF Initialized
                               LatentHeat/Fluxnet Initialized
                                  LatentHeat/GBAF Initialized
                                       Runoff/Dai Initialized
                             SensibleHeat/Fluxnet Initi